In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import os

import re
import random
from termcolor import colored
# import dataframe_image as dfi
# import warnings
# import wandb
# warnings.filterwarnings('ignore')
# warnings.warn('DelftStack')
# warnings.warn('Do not show this message')

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb

from IPython.display import Audio, display
def allDone():
    display(Audio(url='https://www.mediacollege.com/downloads/sound-effects/beep/beep-10.wav', autoplay=True))

embed_path = 'data_test'
result_path = 'ML_predicted_results'
wt_mt_path = 'data_test/wt_mt'

/Users/acetylcoa/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [62]:
# majority and mean prediction votes
def majority_vote(model1, model2, model3, mode_test):
    pred1 = model1.predict_proba(mode_test)
    pred2 = model2.predict_proba(mode_test)
    pred3 = model3.predict_proba(mode_test)
    
    # mean
    stack = pred1 + pred2 + pred3
    stack_mean = stack/3
    mean_vote = [np.argmax(i) for i in stack_mean]

    major_binary_1 = [np.argmax(pred1[i]) for i in range(len(pred1))]
    major_binary_2 = [np.argmax(pred2[i]) for i in range(len(pred2))]
    major_binary_3 = [np.argmax(pred3[i]) for i in range(len(pred3))]

    major_vote = []
    for i in range(len(result_1)):
        tmp_result = []
        tmp_result.append(major_binary_1[i])
        tmp_result.append(major_binary_2[i])
        tmp_result.append(major_binary_3[i])
        result = np.mean(tmp_result)
        if result > 0.5:
            major_vote.append(1)
        else:
            major_vote.append(0)
    
    return mean_vote, major_vote

# print out reports
def report_save(y_true, y_pred, path, mode, name, label_names=None, *args, **kv):
    result_path = f'../data/{path}/ML_predicted_results/{mode}_results'
    # print the classification report here
    report = classification_report(y_true, y_pred, target_names=label_names)
    print(colored(f'\n\t\t\t\t *** {name}_report ***:\n\n\n', 'blue', attrs=['bold']), report)
    MCC = matthews_corrcoef(y_true, y_pred)
    print(f"{name} MCC:", MCC)

    # create report dataframe
    report_for_save = classification_report(y_true, y_pred, target_names=label_names, output_dict=True)
    report_csv = pd.DataFrame(report_for_save).transpose()
    report_csv['MCC'] = MCC

    # style.background_gradient or highlight_max
    report_styled = report_csv.style.background_gradient(subset=['precision', 'recall', 'f1-score'])
    
    # Save results
    if not os.path.isdir(f'{result_path}'):
        os.mkdir(f'{result_path}')

    # export dataframe to .png
    # dfi.export(report_styled, f'{result_path}/{name}_report.png')

    report_csv.to_csv(f'{result_path}/{name}_report_save.csv')


In [32]:
def train_test_dataset(situation, mode):
    if 'imbalance' in situation:
        mode_train = np.load(f'../data/{situation}/For_ML/{mode}_train.npy')
        mode_test = np.load(f'../data/{situation}/For_ML/{mode}_test.npy')

        X_train = mode_train[:,:-1]
        y_train = mode_train[:,-1].astype(int).tolist()
        X_test = mode_test[:,:-1]
        y_test = mode_test[:,-1].astype(int).tolist()
        
        return X_train, y_train, X_test, y_test
    
    else:
        mode_train1 = np.load(f'../data/{situation}/For_ML/{mode}_train_1.npy')
        mode_train2 = np.load(f'../data/{situation}/For_ML/{mode}_train_2.npy')
        mode_train3 = np.load(f'../data/{situation}/For_ML/{mode}_train_2.npy')
        mode_test = np.load(f'../data/{situation}/For_ML/{mode}_test.npy')
        
        X_train1, X_train2, X_train3 = mode_train1[:,:-1], mode_train2[:,:-1], mode_train3[:,:-1]
        y_train1, y_train2, y_train3 = mode_train1[:,-1].astype(int).tolist(), mode_train2[:,-1].astype(int).tolist(), mode_train3[:,-1].astype(int).tolist()
        X_test, y_test = mode_test[:,:-1], mode_test[:,-1].astype(int).tolist()
        
        
        return X_train1, X_train2, X_train3, y_train1, y_train2, y_train3, X_test, y_test

In [47]:
def ML_pred_for_imbalance(X_train, y_train, X_test, y_test, path, mode):
    import lightgbm as lgb
    # XGBoost
    # xgb = XGBClassifier(n_estimators = 300, tree_method='gpu_hist', gpu_id=0)
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)
    y_xgb = xgb.predict(X_test)
    report_save(y_test, y_xgb,path,mode = mode, name =  'XGBoost')
    
    # Catboost
    cbt = CatBoostClassifier(iterations=500, learning_rate=0.09, depth=10)
    cbt.fit(X_train, y_train)
    y_cbt = cbt.predict(X_test)
    report_save(y_test, y_cbt,path, mode = mode, name = 'CatBoost')
    
    # LightGBM
    d_train=lgb.Dataset(X_train, label=y_train)
    params={}
    # params['learning_rate']=0.41282313322582176
    params['learning_rate']=0.1
    params['boosting_type']='gbdt' #GradientBoostingDecisionTree
    params['objective']='cross_entropy' #Binary target feature
    params['metric']='binary_error' #metric for binary classification
    params['max_depth']= 10
    params['n_estimators'] = 300
    # params['num_leaves'] = 34
    # params['reg_lambda'] = 0.9557019573592245
    # params['colsample_by_tree'] = 0.8506663985944544
    lgb = lgb.train(params,d_train,300)
    y_lgb = lgb.predict(X_test)
    y_lgb=y_lgb.round(0)
    y_lgb = y_lgb.astype(int)
    report_save(y_test, y_lgb,path,mode = mode, name = 'LightGBM')
    

In [56]:
def ML_pred_for_balance(X_train1, X_train2, X_train3, y_train1, y_train2, y_train3, X_test, y_test, path, mode):
    
    # XGBoost
    # xgb_1 = XGBClassifier(n_estimators = 300, tree_method='gpu_hist', gpu_id=0)
    xgb_1 = XGBClassifier()
    xgb_2 = XGBClassifier()
    xgb_3 = XGBClassifier()
    xgb_1.fit(X_train1, y_train1)
    xgb_2.fit(X_train2, y_train2)
    xgb_3.fit(X_train3, y_train3)
    xgb_mean_prodictions_binary, xgb_majority_vote_binary = majority_vote(xgb_1,xgb_2,xgb_3, X_test)
    report_save(xgb_mean_prodictions_binary, y_test, path, mode,'XGBoost_mean')
    report_save(xgb_majority_vote_binary, y_test, path, mode,'XGBoost_majority')

    # Catboost
    cbt_1 = CatBoostClassifier(iterations=500, learning_rate=0.09, depth=10)
    cbt_2 = CatBoostClassifier(iterations=500, learning_rate=0.09, depth=10)
    cbt_3 = CatBoostClassifier(iterations=500, learning_rate=0.09, depth=10)

    cbt_1.fit(X_train1, y_train1)
    cbt_2.fit(X_train2, y_train2)
    cbt_3.fit(X_train3, y_train3)
    cbt_mean_prodictions_binary, cbt_majority_vote_binary = majority_vote(cbt_1,cbt_2,cbt_3, X_test)
    report_save(cbt_mean_prodictions_binary, y_test,path,mode, 'Catboost_mean')
    report_save(cbt_majority_vote_binary, y_test,path, mode,'Catboost_majority')

    # LightGBM
    params={}
    params['learning_rate']=0.1
    params['boosting_type']='gbdt' #GradientBoostingDecisionTree
    params['objective']='cross_entropy' #Binary target feature
    params['metric']='binary_error' #metric for binary classification
    params['max_depth']= 10
    params['n_estimators'] = 300
    params['n_iter'] = 500

    lgb_1 = lgb.LGBMClassifier(**params)
    lgb_2 = lgb.LGBMClassifier(**params)
    lgb_3 = lgb.LGBMClassifier(**params)

    lgb_1.fit(X_train1, y_train1)
    lgb_2.fit(X_train2, y_train2)
    lgb_3.fit(X_train3, y_train3)

    lgb_mean_prodictions_binary, lgb_majority_vote_binary = majority_vote(lgb_1,lgb_2,lgb_3, X_test)
    report_save(lgb_mean_prodictions_binary, y_test,path, mode, 'LightGBM_mean')
    report_save(lgb_majority_vote_binary, y_test,path, mode, 'LightGBM_majority')

## Situation_1
imbalance + same seq, Train:Test = 3:1

In [35]:
X1_train_s1, y1_train_s1, X1_test_s1, y1_test_s1 = train_test_dataset('imbalance_same_seq', 'mode1')

X2_train_s1, y2_train_s1, X2_test_s1, y2_test_s1 = train_test_dataset('imbalance_same_seq', 'mode2')


In [36]:
X1_train_s1.shape, len(y1_train_s1)

((22200, 2048), 22200)

In [49]:
# X1_train_s1, y1_train_s1, X1_test_s1, y1_test_s1 = X1_train_s1[:100,:], y1_train_s1[:100], X1_test_s1[:100,:], y1_test_s1[:100]
# X2_train_s1, y2_train_s1, X2_test_s1, y2_test_s1 = X2_train_s1[:100,:], y2_train_s1[:100], X2_test_s1[:100,:], y2_test_s1[:100]


In [38]:
X1_train_s1.shape

(100, 2048)

In [48]:
ML_pred_for_imbalance(X1_train_s1, y1_train_s1, X1_test_s1, y1_test_s1,'imbalance_same_seq', mode = 'mode1')
ML_pred_for_imbalance(X2_train_s1, y2_train_s1, X2_test_s1, y2_test_s1,'imbalance_same_seq', mode = 'mode2')

/Users/acetylcoa/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:44:06] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

				 *** XGBoost_report ***:


               precision    recall  f1-score   support

           0       0.76      0.97      0.85        74
           1       0.60      0.12      0.19        26

    accuracy                           0.75       100
   macro avg       0.68      0.54      0.52       100
weighted avg       0.72      0.75      0.68       100

0:	learn: 0.6343911	total: 166ms	remaining: 8.13s
1:	learn: 0.5874617	total: 334ms	remaining: 8.02s
2:	learn: 0.5436758	total: 553ms	remaining: 8.66s
3:	learn: 0.5032461	total: 809ms	remaining: 9.3s
4:	learn: 0.4705578	total: 1.02s	remaining: 9.15s
5:	learn: 0.4374428	total: 1.21s	remaining: 8.89s
6:	learn: 0.4123554	tota

/Users/acetylcoa/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] [cross_entropy:BoostFromScore]: pavg = 0.200000 -> initscore = -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/Users/acetylcoa/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:44:24] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

				 *** XGBoost_report ***:


               precision    recall  f1-score   support

           0       0.79      0.91      0.84        75
           1       0.50      0.28      0.36        25

    accuracy                           0.75       100
   macro avg       0.65      0.59      0.60       100
weighted avg       0.72      0.75      0.72       100

0:	learn: 0.6385082	total: 186ms	remaining: 9.11s
1:	learn: 0.5946046	total: 376ms	remaining: 9.02s
2:	learn: 0.5509072	total: 567ms	remaining: 8.88s
3:	learn: 0.5115692	total: 765ms	remaining: 8.79s
4:	learn: 0.4784691	total: 959ms	remaining: 8.63s
5:	learn: 0.4440462	total: 1.07s	remaining: 7.85s
6:	learn: 0.4132627	tot

/Users/acetylcoa/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.170996 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71680
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 2048
[LightGBM] [Info] [cross_entropy:BoostFromScore]: pavg = 0.280000 -> initscore = -0.944462
[LightGBM] [Info] Start training from score -0.944462
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

## Situation 2

In [64]:
X1_train1_s2, X1_train2_s2, X1_train3_s2, y1_train1_s2, y1_train2_s2, y1_train3_s2, X1_test_s2, y1_test_s2 = train_test_dataset('balanced_same_seq', 'mode1')
X2_train1_s2, X2_train2_s2, X2_train3_s2, y2_train1_s2, y2_train2_s2, y2_train3_s2, X2_test_s2, y2_test_s2 = train_test_dataset('balanced_same_seq', 'mode2')


In [65]:
# X1_train1_s2, X1_train2_s2, X1_train3_s2, y1_train1_s2, y1_train2_s2, y1_train3_s2, X1_test_s2, y1_test_s2 = X1_train1_s2[:100,:], X1_train2_s2[:100,:], X1_train3_s2[:100,:], y1_train1_s2[:100], y1_train2_s2[:100], y1_train3_s2[:100], X1_test_s2[:100,:], y1_test_s2[:100]
# X2_train2_s2, X2_train2_s2, X2_train3_s2, y2_train2_s2, y2_train2_s2, y2_train3_s2, X2_test_s2, y2_test_s2 = X2_train2_s2[:100,:], X2_train2_s2[:100,:], X2_train3_s2[:100,:], y2_train2_s2[:100], y2_train2_s2[:100], y2_train3_s2[:100], X2_test_s2[:100,:], y2_test_s2[:100]


In [66]:
from collections import Counter
Counter(y1_train2_s2)

Counter({1: 57, 0: 43})

In [63]:
ML_pred_for_balance(X1_train1_s2, X1_train2_s2, X1_train3_s2, y1_train1_s2, y1_train2_s2, y1_train3_s2, X1_test_s2, y1_test_s2, 'balanced_same_seq', 'mode1')
ML_pred_for_balance(X2_train1_s2, X2_train2_s2, X2_train3_s2, y2_train1_s2, y2_train2_s2, y2_train3_s2, X2_test_s2, y2_test_s2, 'balanced_same_seq', 'mode2')


[16:12:31] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:12:31] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:12:31] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[1 0]
[[1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 

ValueError: Found input variables with inconsistent numbers of samples: [2, 100]

## Situation 3

In [70]:
X1_train_s3, y1_train_s3, X1_test_s3, y1_test_s3 = train_test_dataset('imbalance_diff_seq', 'mode1')
X2_train_s3, y2_train_s3, X2_test_s3, y2_test_s3 = train_test_dataset('imbalance_diff_seq', 'mode2')

In [ ]:
ML_pred_for_imbalance(X1_train_s3, y1_train_s3, X1_test_s3, y1_test_s3,'imbalance_diff_seq', 'mode1')
ML_pred_for_imbalance(X2_train_s3, y2_train_s3, X2_test_s3, y2_test_s3,'imbalance_diff_seq', 'mode2')

## Situation 4

In [77]:
X1_train1_s4, X1_train2_s4, X1_train3_s4, y1_train1_s4, y1_train2_s4, y1_train3_s4, X1_test_s4, y1_test_s4 = train_test_dataset('balanced_diff_seq', 'mode1')
X2_train1_s4, X2_train2_s4, X2_train3_s4, y2_train1_s4, y2_train2_s4, y2_train3_s4, X2_test_s4, y2_test_s4 = train_test_dataset('balanced_diff_seq', 'mode2')


In [81]:
len(X2_train1_s4)

13193

In [ ]:
ML_pred_for_balance(X1_train1_s4, X1_train2_s4, X1_train3_s4, y1_train1_s4, y1_train2_s4, y1_train3_s4, X1_test_s4, y1_test_s4, 'balanced_diff_seq','mode1')
ML_pred_for_balance(X2_train1_s4, X2_train2_s4, X2_train3_s4, y2_train1_s4, y2_train2_s4, y2_train3_s4, X2_test_s4, y2_test_s4, 'balanced_diff_seq','mode2')
